In [1]:
# import the dependencies
import pandas as pd
import requests
import gmaps
from config import g_key

In [2]:
# store the CSV you saved created in part one into a dataframe
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Upernavik,GL,2021-03-24 18:24:34,72.7868,-56.1549,-13.34,86,100,3.83
1,1,Hamilton,US,2021-03-24 18:22:36,39.1834,-84.5333,72.00,49,75,14.97
2,2,Rundu,NaN,2021-03-24 18:24:34,-17.9333,19.7667,74.25,67,99,4.45
3,3,Georgetown,MY,2021-03-24 18:24:35,5.4112,100.3354,80.01,94,20,1.14
4,4,Port Elizabeth,ZA,2021-03-24 18:24:35,-33.9180,25.5701,69.80,83,20,11.50


In [3]:
# get the datatypes
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# configure gmaps to use your api key
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity =300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig 
                       

Figure(layout=FigureLayout(height='420px'))

In [6]:
# heatmap of percent humidity
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# heat map of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# wind speed heat map
locations = city_data_df[["Lat", "Lng"]]
wind= city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?" ))
max_temp = float(input("What is the maximum temperature you would like for your trip?" ))


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/isabellaferaca/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-fff3bf877515>", line 2, in <module>
    min_temp = float(input("What is the minimum temperature you would like for your trip?" ))
  File "/Users/isabellaferaca/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 863, in raw_input
    password=False,
  File "/Users/isabellaferaca/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 904, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/isabellaferaca/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/IPython/core/inter

TypeError: object of type 'NoneType' has no len()

In [ ]:
# filter the dataset to find the citites that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
preferred_cities_df.count()

In [ ]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# set parameters to search for a hotel
params = {
    "radius":5000,
    "type":"lodging",
    "key": g_key
}

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")

In [ ]:
hotel_df.head(10)

In [ ]:
# add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.03,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# call the figure to plot the data
fig

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# store the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig